#Adding required Libraries

In [ ]:
#pip install emoji

     |████████████████████████████████| 170 kB 8.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=b9bd826dda33324a9a44735257b9f09314f9f8c731e861b0b7c4bd4abc31c2aa
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import numpy as np
import pandas as pd
import re,string
#import emoji
from google.colab import drive

#Uploading Data

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATADIR = "gdrive/MyDrive/DataSets/Coronavirus tweets NLP - Text Classification"
train_data = pd.read_csv(DATADIR + "/Corona_NLP_train.csv",encoding="latin-1")
test_data = pd.read_csv(DATADIR + "/Corona_NLP_test.csv",encoding="latin-1")

#Understanding Data

In [ ]:
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
test_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       3798 non-null   int64 
 1   ScreenName     3798 non-null   int64 
 2   Location       2964 non-null   object
 3   TweetAt        3798 non-null   object
 4   OriginalTweet  3798 non-null   object
 5   Sentiment      3798 non-null   object
dtypes: int64(2), object(4)
memory usage: 178.2+ KB


In [ ]:
train_data.isnull().sum(axis = 0)

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [ ]:
test_data.isnull().sum(axis = 0)

UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64

In [ ]:
train_data["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
test_data["Sentiment"].value_counts()

Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64

In [ ]:
for i in range(20):
  print("Tweet{}\tSentiment: {} \n{}\n".format(i+1,train_data["Sentiment"][i],train_data["OriginalTweet"][i]))

Tweet1	Sentiment: Neutral 
@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8

Tweet2	Sentiment: Positive 
advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order

Tweet3	Sentiment: Positive 
Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P

Tweet4	Sentiment: Positive 





#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j

Tweet5	Sentiment: Extremely Negative 




#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n

Tweet6	Sentiment: Positive 
As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area store

#Cleaning Data

In [ ]:
train_data = train_data[["OriginalTweet","Sentiment"]]
test_data = test_data[["OriginalTweet","Sentiment"]]

In [ ]:
train_data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
test_data.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


##Cleaning Duplicate tweets

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
dtypes: object(2)
memory usage: 643.2+ KB


In [ ]:
train_data.drop_duplicates(subset="OriginalTweet",inplace=True)

In [ ]:
train_data.info() #No duplicate tweets.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
dtypes: object(2)
memory usage: 964.6+ KB


#Normalization Test1



*   Remove all mentions and URLs
*   Remove all emojis
*   Remove all \n and \r
*   Remove all special characters (&,$,£,¥)
*   Remove end hashtags at the and keep middle hashtags without # symbol
*   Remove multiple spaces
*   Lowercase

In [ ]:
def remove_mention_URL(tweet):
  return re.sub(r"(?:\@|https?\://)\S+", "", tweet)
  

In [ ]:
test_tweet = "@aydcsr Kanka şu linke baksana https://www.youtube.com/watch?v=0obBdrfUMzU"
test_tweet = remove_mention_URL(test_tweet)
print(test_tweet),
test_data_tweet = remove_mention_URL(train_data["OriginalTweet"][0])
print(train_data["OriginalTweet"][0])
print(test_data_tweet)

 Kanka şu linke baksana 
@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
    and  and 


In [ ]:
#for tweet in train_data["OriginalTweet"][:500]:
#  print(tweet.encode("latin-1").decode("utf8"))

No emoji was found when looking at the tweets. That's why emoji normalization is skipped for now.

In [ ]:
def remove_newline(tweet):
  return tweet.replace('\r', '').replace('\n', ' ')
test_tweet = "Kanka\nŞuna\n\rbi baksana\raloo"
print(test_tweet)
test_tweet = remove_newline(test_tweet)
print("After :",test_tweet)

Kanka
Şuna
aloo
After : Kanka Şuna bi baksanaaloo


In [ ]:
def remove_char(tweet):
  return tweet.replace('$',' ').replace('£',' ').replace('&',' ').replace('¥',' ')
test_tweet = "Dolar$ : 18.5\nEuro$: 20 Küsür\nYen¥: Yenmeyecek seviyede & Bu kötü bir şaka"
print(test_tweet)
test_tweet = remove_char(test_tweet)
print("After:\n",test_tweet)

Dolar$ : 18.5
Euro$: 20 Küsür
Yen¥: Yenmeyecek seviyede & Bu kötü bir şaka
After:
 Dolar  : 18.5
Euro : 20 Küsür
Yen : Yenmeyecek seviyede   Bu kötü bir şaka


In [ ]:
def remove_hashtag(tweet):
  new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
  new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
  return new_tweet2
test_tweet = "#HT1 burada bazı yazılar #HT2 Burada da bazı yazılar #HT3 #HT4"
print(test_tweet)
test_tweet = remove_hashtag(test_tweet)
print("After\n",test_tweet)

#HT1 burada bazı yazılar #HT2 Burada da bazı yazılar #HT3 #HT4
After
  HT1 burada bazı yazılar HT2 Burada da bazı yazılar


In [ ]:
def remove_mult_spaces(tweet):
    return re.sub("\s\s+" , " ", tweet)
test_tweet = "Şimdi  ben    buraya ne   yazıcam    niye    yazıcam"
print(test_tweet)
test_tweet = remove_mult_spaces(test_tweet)
print("After\n",test_tweet)

Şimdi  ben    buraya ne   yazıcam    niye    yazıcam
After
 Şimdi ben buraya ne yazıcam niye yazıcam


In [ ]:
def normalize(tweet):
  tweet = remove_mention_URL(tweet)
  tweet = remove_newline(tweet)
  tweet = remove_char(tweet)
  tweet = remove_hashtag(tweet)
  tweet = remove_mult_spaces(tweet)
  tweet = tweet.lower()
  return tweet

In [ ]:
test_tweet = "@aydcsr\r #Kanka\n£al $al  YATIRIM YAP\n #SaveMoney @user123 #COVID"
print(test_tweet)
test_tweet = normalize(test_tweet)
print("After\n",test_tweet)

 #Kanka
£al $al  YATIRIM YAP
 #SaveMoney @user123 #COVID
After
  kanka al al yatirim yap


Şuana kadar güzel iş yapıyor bence

##Normalize train and test data

In [ ]:
new_train_data = []
for tweet in train_data["OriginalTweet"]:
  new_tweet = normalize(tweet)
  new_train_data.append(new_tweet)

In [ ]:
new_test_data = []
for tweet in test_data["OriginalTweet"]:
  new_tweet = normalize(tweet)
  new_test_data.append(new_tweet)

In [ ]:
new_test_data[0]

'trending: new yorkers encounter empty supermarket shelves (pictured, wegmans in brooklyn), sold-out online grocers (foodkick, maxdelivery) as coronavirus-fearing shoppers stock up'

In [ ]:
train_tweets_lens = []
for tweet in new_train_data:
  tweet_len = len(tweet.split())
  train_tweets_lens.append(tweet_len)

In [ ]:
test_tweets_lens = []
for tweet in new_test_data:
  tweet_len = len(tweet.split())
  test_tweets_lens.append(tweet_len)

In [ ]:
new_train_df = {"Tweet":new_train_data, "Tweet_len":train_tweets_lens,"Sentiment":train_data["Sentiment"]}
if len(new_train_df["Tweet"])!=len(new_train_df["Tweet_len"]) or len(new_train_df["Tweet"])!=len(new_train_df["Sentiment"]):
  print("Problem var")
else:
  print("All OK")

All OK


In [ ]:
for i in range(len(new_train_df["Tweet"])):
  if new_train_df["Tweet_len"][i] < 5:
    print(new_train_df["Tweet"][i])
  else:
    pass


and and
dear hoarder : covid?19
cleanshelf supermarket sanitizers 19

please read this thread
binge shopping online
here's the full list:
student resources during covid-19
clean-up on aisle 3:16.
this is kind
time to step up
this is just madness!!
for more updates, visit
online shopping
 ny wilding rn ??
covid-19 supermarket sweep
covid-19 retail store closures
âthe screamâ mk 2020

grocery store this morning
checks from the government
the poor supermarket workers
another update
meantime, at
release update information.
england 2020 coronavirus covid?19uk
managing covid-19 disruption
 coronavirus increases chinese phones
 gas during coronavirus, from
chain retail reaction
oil prices at 23.18
oil prices at 22.90
updated product limits from
a few wise words.
please stop panic buying!
supermarket owners rn ??
?? supermarket job openings
the new supermarket wars
brings to its knees
empty supermarket aisles everywhere
expose these greedy bastards.

from today's
it has begun.
so, calm down,

... ile kelimeleri ayırmışlar bu ayrımı da ilerleyen zamanlarda normalize etmeliyiz

Gelecek Çalışmada Yapılacaklar.

* tweet uzunluğu 5 den küçük olanları datasetten çıkar  
* normalize edilmiş train ve test datalarını üzerinde çalışmak için kaydet
* Bert modeli için bu verileri test et

